In [240]:
%gui osx
import py5
import numpy as np
import random

In [241]:
# Given three collinear points p, q, r, the function checks if 
# point q lies on line segment 'pr' 
def onSegment(p, q, r):
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))):
        return True
    return False
  
def orientation(p, q, r):
    # to find the orientation of an ordered triplet (p,q,r)
    # function returns the following values:
    # 0 : Collinear points
    # 1 : Clockwise points
    # 2 : Counterclockwise
      
    # See https://www.geeksforgeeks.org/orientation-3-ordered-points/amp/ 
    # for details of below formula. 
      
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y))
    if (val > 0):
          
        # Clockwise orientation
        return 1
    elif (val < 0):
          
        # Counterclockwise orientation
        return 2
    else:
          
        # Collinear orientation
        return 0
  
# The main function that returns true if 
# the line segment 'p1q1' and 'p2q2' intersect.
def doIntersect(p1,q1,p2,q2):
      
    # Find the 4 orientations required for 
    # the general and special cases
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
  
    # General case
    if ((o1 != o2) and (o3 != o4)):
        return True
  
    # Special Cases
  
    # p1 , q1 and p2 are collinear and p2 lies on segment p1q1
    if ((o1 == 0) and onSegment(p1, p2, q1)):
        return True
  
    # p1 , q1 and q2 are collinear and q2 lies on segment p1q1
    if ((o2 == 0) and onSegment(p1, q2, q1)):
        return True
  
    # p2 , q2 and p1 are collinear and p1 lies on segment p2q2
    if ((o3 == 0) and onSegment(p2, p1, q2)):
        return True
  
    # p2 , q2 and q1 are collinear and q1 lies on segment p2q2
    if ((o4 == 0) and onSegment(p2, q1, q2)):
        return True
  
    # If none of the cases
    return False
   

In [242]:
class mPoint:
    def __init__(self,inx, iny, index=-1, szomszedok={}):
        self.x =inx
        self.y =iny
        self.index=index
        self.szomszedok=szomszedok

        
    def __add__(self, other):
        return mPoint(self.x+other.x,self.y+other.y)
    
    def __sub__(self, other):
        return mPoint(self.x-other.x,self.y-other.y)
    
    def __str__(self):
        return str('('+str(self.index)+','+str(self.x)+','+str(self.y)+','+str(self.szomszedok)+')')
    
    def mag(self):
        return np.sqrt(self.magSq())
  
    def magSq(self):
        return self.x*self.x+self.y*self.y;
    
    def display(self):
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
    
    def display_elestol(self):
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
        py5.stroke_weight(4)
        for csucs in self.szomszedok:
            py5.line(self.x,self.y,graf_csucsai.plist[csucs].x,graf_csucsai.plist[csucs].y)


class edge:
    global graf_elei
    def __init__(self, inStart,inEnd):
        self.s =inStart
        self.t =inEnd
        self.metszo=False
       
    def display(self):
        py5.stroke_weight(4)
        if self.metszo==True:
            py5.stroke(204, 102, 0)
        else:
            py5.stroke(153)
            
        py5.line(self.s.x,self.s.y,self.t.x,self.t.y)
        
    def metsz_e_valamit(self):
        ideig=False
        for el in graf_elei.plist:
            if doIntersect(self.s,self.t,el.s,el.t):
                ideig=True
        self.metszo=ideig
        return True
   
class edge_set:
    def __init__(self, plist=[]):
        self.plist=plist
        
    def display(self):
        for x in self.plist:
            x.display()
    def van_e_metszes(self):
        for x in self.plist:
            x.metsz_e_valamit()
        
class dinamic_pointset:
    def __init__(self, plist=[]):
        self.plist=plist
        self.active=False
        self.act_point=mPoint(0,0,0,[])
        self.log=False         
       
    def display(self):
        for x in self.plist:
            x.display()
    
    def select(self):
        if self.log:
            print('select')
        
        ref_point=mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)
        ref_point.display()
        dist_bound=30
        min_dist=dist_bound
        min_point=mPoint(0,0)
        for p in self.plist:
            if dist(ref_point,p)<min_dist:
                min_dist=dist(ref_point,p)
                min_point=p
        
        if min_dist<dist_bound:
            if self.log:
                print('succes select')
        
            self.active=True
            self.act_point=min_point
        else:
            if self.log:
                print('not succes select')
     #   self.plist.append(ref_point)
        
    def update(self):
        #print('update')
        if self.active:
            self.act_point.x=py5.mouse_x-py5.width/2
            self.act_point.y=py5.mouse_y-py5.height/2
    
    def relase(self):
        if self.log:
            print('re')
        
        self.active=False
   
    def do_something(self):
        if self.active:
            self.relase()
        else:
            self.select()
    
    def add_point(self,p):
        if not self.active:
            self.plist.append(p)
        else:
            if self.log:
                print('cant add now, point set is active')
 
def dist( a, b):
    return (a-b).mag();           
            
def mouse_point():
    return mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)

In [243]:
def random_adjacencia_matrix():
    matrix=np.random.randint(2,size=(10,10))
    for i in range(10):
        for j in range(10):
            if i==j:
                matrix[i][j]=0
            elif i<j:
                matrix[i][j]=matrix[j][i]
    return matrix

In [244]:
def setup():
    py5.size(1000, 1000)
    py5.rect_mode(py5.CENTER)

num_of_points=10
points=np.random.rand(num_of_points,2)*1000-500
adjacencia_matrix=random_adjacencia_matrix()
ideig_ponthalmaz=[]
elek=[]
i=0

for x,y in points:
    random_csucshalmaz=set(j for j in range(10) if adjacencia_matrix[i][j]==1)
    ideig_ponthalmaz.append(mPoint(x,y,i,random_csucshalmaz))
    i+=1
graf_csucsai=dinamic_pointset(ideig_ponthalmaz)

ideiglenes_elhalmaz=[]
for i in range(10):
    for j in range(10):
        if adjacencia_matrix[i][j]==1:
            ideiglenes_elhalmaz.append(edge(graf_csucsai.plist[i],graf_csucsai.plist[j]))
            
graf_elei=edge_set(ideiglenes_elhalmaz)

def mouse_released(e):
    global graf_csucsai
    
    modifiers = e.get_modifiers()
    msgs = []
    if modifiers & e.SHIFT:
        graf_csucsai.add_point(mouse_point())
    else:
        graf_csucsai.do_something()

        

In [245]:
for point in graf_csucsai.plist:
    print(point)
    

(0,-57.93522020478582,-229.38041291832423,{1, 2, 4, 6, 7, 8})
(1,-54.39308378336017,-495.49667820595187,{0, 9, 5, 7})
(2,306.11627141868007,-415.73067358979034,{0, 4, 5, 7, 8})
(3,429.28645820363647,-232.32237332766624,{9, 4, 5})
(4,-430.1822204988802,-289.3428577684407,{0, 8, 2, 3})
(5,-419.51984748886906,210.11742308020757,{1, 2, 3, 6, 8})
(6,272.7337276758626,-401.156794058532,{0, 8, 5, 7})
(7,295.70877644807,318.8789864701887,{0, 1, 2, 6, 8})
(8,204.51578087594316,379.8217265400368,{0, 2, 4, 5, 6, 7, 9})
(9,-78.72942745333933,404.35575149952683,{8, 1, 3})


In [246]:
len(graf_csucsai.plist)

10

In [247]:
    
def draw():
    # adatok
    
    global graf_csucsai
    mmouse_x=py5.mouse_x-py5.width/2
    mmouse_y=py5.mouse_y-py5.height/2
    graf_csucsai.update()
    graf_elei.van_e_metszes()
    
    #stílus
    py5.background(255)
    py5.translate(py5.width/2, py5.height/2);
    
    #rajzolás
    
    py5.stroke(0)
    py5.stroke_weight(10)
    graf_csucsai.display()
    graf_elei.display()
    
    points=graf_csucsai.plist
    n=len(graf_csucsai.plist)
    points_raw=[(p.x,p.y) for p in graf_csucsai.plist]
    
    
py5.run_sketch()